# The `tenacity` Library

This library is used to retry code under certain conditions.

There are many times we need to retry some code that failed to execute as expected.

For example, you might be querying an API and hit a timeout, or a rate limit - in those cases you usually want to retry the call, possibly with some delay, and often we want to retry more than once.

In a previous video I showed how to write a custom decorator to implement retries on API calls.

This library greatly simplifies the process with many additional features, and of course applies to many other use cases too.

The `tenacity` library can be pip installed using:
```bash
pip install tenacity
```

The docs for this library are available [here](https://tenacity.readthedocs.io/en/latest/)

In [1]:
import tenacity

Let's start with a simple example.

By the way, before anyone flames me in the comments, I know this is a **terrible** design for this problem (at the very least I would use an auto incrementing default row id, a UUID, and probably a dozen other ways). I am just trying to show how this `tenacity` library works, for applications beyond calling APIs (which I'll show later).

Suppose we have a system that maintains records for companies. Each company has it's own unique ID, but that ID needs to be a random integer, not a UUID. The problem is then how to generate a new ID that does not already exist.

To simulate sych a system, I'm going to create a sqlite table, add a few records in there. To make the example easier to reproduce, I'm going to limit the range of random IDs to 1-100, and pre-populate our database with some records.

First we establish a connection to a database (which I will set up as an in-memory-only database), and get a "cursor" (an object we use to communicate with the database).

In [2]:
import sqlite3

db_conn = sqlite3.connect(":memory:")
db_cur = db_conn.cursor()

> Note: if you want an actual persistent database, just specify a file name instead of `:memory:`, e.g. : `sqlite3.connect("some/path/my_database.db")`

Next, we create a simple table with an ID and a name for our widgets:

In [3]:
db_cur.execute("""
create table companies(id int primary key, name text);
""")

We can easily query for records in this table like this:

In [4]:
results = db_cur.execute("select * from companies;")

print(results.fetchall())

[]


As expected our table is empty, but we at least can see that the table now exists.

Let's add some data to it. To do this I'm going to generate 50 company records by using random integers between 1 and 100 as the ID, and using the [Faker](https://faker.readthedocs.io/en/master/) library I covered in a previous video.

In [5]:
import random

from faker import Faker
from faker.providers import company


def generate_fake_companies(n=50):
    fake = Faker()
    fake.add_provider(company)

    company_ids = range(1, 101)
    return [
        {
            "id": company_id,
            "name": fake.company()
        }
        for company_id in random.sample(company_ids, n)
    ]

random.seed(0)
Faker.seed(0)
companies = generate_fake_companies(50)
print(companies[:5])

[{'id': 50, 'name': 'Chang-Fisher'}, {'id': 98, 'name': 'Sheppard-Tucker'}, {'id': 54, 'name': 'Faulkner-Howard'}, {'id': 6, 'name': 'Wagner LLC'}, {'id': 34, 'name': 'Campos PLC'}]


Ok, so now it's time to save these companies to our database.

In [6]:
db_cur.executemany("insert into companies(id, name) values(:id, :name);", companies)

And let's make sure the data was written to our database:

In [7]:
result = db_cur.execute("select count(*) from companies;")
print(result.fetchone())

(50,)


And to retrieve a single record we can do it this way (there are other ways, but let's keep it simple so we can focus on `tenacity` instead of sqlite).

In [8]:
result = db_cur.execute("select * from companies where id = ?;", (50, ))
print(result.fetchone())

(50, 'Chang-Fisher')


One thing to note is that we made `id` a primary key, which means the database will reject an insert if there is a conflict:

In [9]:
try:
    db_cur.execute("insert into companies (id, name) values(50, 'test');")
except sqlite3.IntegrityError as ex:
    print(ex)

UNIQUE constraint failed: companies.id


Now that we have our database and some pre-populated data, let's write a function that will create a new company and insert it into the database. The catch, is we want the function to assign a random ID between 1 and 100 to the new record.

In [10]:
def create_company(cur, company_name):
    company_id = random.randint(1, 100)
    print(f"creating company with id: {company_id}...")
    cur.execute("insert into companies (id, name) values(?, ?);", (company_id, company_name))
    
    return company_id, company_name

In [11]:
random.seed(0)
create_company(db_cur, "test")

creating company with id: 50...


IntegrityError: UNIQUE constraint failed: companies.id

In [12]:
create_company(db_cur, "test")

creating company with id: 98...


IntegrityError: UNIQUE constraint failed: companies.id

As you can see, we keep failing because the ID we are attempting to use already exists in the database.

So, how do we find a "free" ID? 

There are many ways to do this, but since I'm trying to demonstrate the usefulness of `tenacity` we'll take a brute force approach. We just want to keep repeating our attempts to create a new company (with a random ID) until it actually works.

We could certainly use a `while` loop or other mechanism to do that directly in the code. And if we only need this kind of retry in a single place in our code, maybe that's the simplest.

But consider how you can do this using `tenacity` and think of where it might be useful in other scenarios.

We could simply retry based on any exception, like this:

## Basic Retry

In [13]:
@tenacity.retry
def create_company(cur, company_name):
    company_id = random.randint(1, 100)
    print(f"creating company with id: {company_id}...")
    cur.execute("insert into companies (id, name) values(?, ?);", (company_id, company_name))
    
    return company_id, company_name

In [14]:
random.seed(0)
create_company(db_cur, "test")

creating company with id: 50...
creating company with id: 98...
creating company with id: 54...
creating company with id: 6...
creating company with id: 34...
creating company with id: 66...
creating company with id: 63...
creating company with id: 52...
creating company with id: 39...
creating company with id: 62...
creating company with id: 46...
creating company with id: 75...
creating company with id: 28...
creating company with id: 65...
creating company with id: 18...
creating company with id: 37...
creating company with id: 18...
creating company with id: 97...
creating company with id: 13...
creating company with id: 80...
creating company with id: 33...
creating company with id: 69...
creating company with id: 91...
creating company with id: 78...
creating company with id: 19...
creating company with id: 40...
creating company with id: 13...
creating company with id: 94...


(94, 'test')

You can see how the function was retried until it found an "available" ID.

## Retrying Based on Specific Exception Types

But what if other exceptions in our function, that have nothing to do with the `IntegrityError` exception - maybe we don't want to retry those, and would instead want to let the exception bubble through and let the caller handle other exceptions.

To do this, we need to specify a retry exception.

Let's try it:

In [15]:
@tenacity.retry(retry=tenacity.retry_if_exception_type(sqlite3.IntegrityError))
def create_company(cur, company_name):
    company_id = random.randint(1, 100)
    print(f"creating company with id: {company_id}...")

    # let's raise a ValueError exception if the ID is odd
    if company_id % 2:
        raise ValueError("An odd number was used for an ID")

    cur.execute("insert into companies (id, name) values(?, ?);", (company_id, company_name))
    
    return company_id, company_name

In [16]:
try:
    random.seed(0)
    create_company(db_cur, "test")
except ValueError as ex:
    print(ex)

creating company with id: 50...
creating company with id: 98...
creating company with id: 54...
creating company with id: 6...
creating company with id: 34...
creating company with id: 66...
creating company with id: 63...
An odd number was used for an ID


As you can see, as long as the exception in our function was an `IntegrityError` the function was retried, but as soon as another exception was encountered (a `ValueError` in this case), the retry stopped, and the exception bubbled up to us (the caller of `create_company()`.

## Setting Retry Delays and Limits

Another thing you may have noticed is that the retries happened one after the other without any delay.

We sometimes want to introduce a delay between retries (usually to give time for some state somewhere to change, maybe a rate limit to reset, a network connection to be restored, etc).

Tenacity has us covered there as well, with many different types of retry waits.

The simplest is just a fixed time (specified in seconds):

In [17]:
@tenacity.retry(
    retry=tenacity.retry_if_exception_type(sqlite3.IntegrityError),
    wait=tenacity.wait_fixed(1)
)
def create_company(cur, company_name):
    company_id = random.randint(1, 100)
    print(f"creating company with id: {company_id}...")

    # let's raise a ValueError exception if the ID is odd
    if company_id % 2:
        raise ValueError("An odd number was used for an ID")

    cur.execute("insert into companies (id, name) values(?, ?);", (company_id, company_name))
    
    return company_id, company_name

In [18]:
try:
    random.seed(0)
    create_company(db_cur, "test")
except ValueError as ex:
    print(ex)

creating company with id: 50...
creating company with id: 98...
creating company with id: 54...
creating company with id: 6...
creating company with id: 34...
creating company with id: 66...
creating company with id: 63...
An odd number was used for an ID


Other wait specifications include random times (within specified limits):

In [19]:
@tenacity.retry(
    retry=tenacity.retry_if_exception_type(sqlite3.IntegrityError),
    wait=tenacity.wait_random(min=0.5, max=2)
)
def create_company(cur, company_name):
    company_id = random.randint(1, 100)
    print(f"creating company with id: {company_id}...")

    # let's raise a ValueError exception if the ID is odd
    if company_id % 2:
        raise ValueError("An odd number was used for an ID")

    cur.execute("insert into companies (id, name) values(?, ?);", (company_id, company_name))
    
    return company_id, company_name

In [20]:
try:
    random.seed(0)
    create_company(db_cur, "test")
except ValueError as ex:
    print(ex)

creating company with id: 50...
creating company with id: 54...
creating company with id: 66...
creating company with id: 39...
An odd number was used for an ID


(If you're wondering why the sequence of id's generated here is different from the one above even though I reset the seed to `0`, that's because that `wait_random` is also using Python's random generator)

When dealing with rate limited APIs, we often use exponential backoffs - this is where the wait times increase exponentially. We usually also limit the number of times we want to retry things.

So let's implement both here:

In [21]:
@tenacity.retry(
    retry=tenacity.retry_if_exception_type(sqlite3.IntegrityError),
    wait=tenacity.wait_exponential(multiplier=1, min=1, max=10),
    stop=tenacity.stop_after_attempt(6)
)
def create_company(cur, company_name):
    company_id = random.randint(1, 100)
    print(f"creating company with id: {company_id}...")

    # let's raise a ValueError exception if the ID is odd
    if company_id % 2:
        raise ValueError("An odd number was used for an ID")

    cur.execute("insert into companies (id, name) values(?, ?);", (company_id, company_name))
    
    return company_id, company_name

The way the exponential backoff works here, is that the first retry waits `min` seconds, the second retry waits `2^1 * 1`, then `2^2 * 1`, up to a maximum wait of `max` seconds.

And we limit the number of retries using `stop_after_attempt`, but we could also use `stop_after_delay` where we basically retry, increasing our delay, until it reaches some value at which we want to stop trying.

In [22]:
try:
    random.seed(0)
    create_company(db_cur, "test")
except ValueError as ex:
    print(ex)

creating company with id: 50...
creating company with id: 98...
creating company with id: 54...
creating company with id: 6...
creating company with id: 34...
creating company with id: 66...


RetryError: RetryError[<Future at 0x11073ed10 state=finished raised IntegrityError>]

As you can see the wait times increased until it reached 10seconds, and finally we received a `RetryError` because we reached the maximum number of attempts. 

To stop after the delay reaches a certain value, we could do this:

In [23]:
@tenacity.retry(
    retry=tenacity.retry_if_exception_type(sqlite3.IntegrityError),
    wait=tenacity.wait_exponential(multiplier=1, min=1, max=10),
    stop=tenacity.stop_after_delay(10)
)
def create_company(cur, company_name):
    company_id = random.randint(1, 100)
    print(f"creating company with id: {company_id}...")

    # let's raise a ValueError exception if the ID is odd
    if company_id % 2:
        raise ValueError("An odd number was used for an ID")

    cur.execute("insert into companies (id, name) values(?, ?);", (company_id, company_name))
    
    return company_id, company_name

In [24]:
from time import perf_counter

start = perf_counter()
try:
    random.seed(0)
    create_company(db_cur, "test")
except ValueError as ex:
    print('ValueError:', ex)
except tenacity.RetryError as ex:
    print('Retry Error:', ex)
end = perf_counter()
print(end - start)

creating company with id: 50...
creating company with id: 98...
creating company with id: 54...
creating company with id: 6...
creating company with id: 34...
Retry Error: RetryError[<Future at 0x110702690 state=finished raised IntegrityError>]
15.023358792008366


## Viewing Retry Stats

As you can see we got our retry error because the total retry time reached over `10` seconds.

We can actually inspect the retry statistics:

In [25]:
create_company.retry.statistics

{'start_time': 96407.2631565,
 'attempt_number': 5,
 'idle_for': 15.0,
 'delay_since_first_attempt': 15.02236937500129}

## Retrying Based on Function Return Value

There are other retry definitions beyond just the exception based one we just saw.

We can use things like the return value of the called function, and retry until some condition is met.

To do so, we specify a predicate function that will receive the function's return value and decide whether to retry (by returning `True`), or not retry (by returning `False`)

For example:

In [26]:
@tenacity.retry(
    retry=tenacity.retry_if_result(lambda result: result is None),
)
def create_company(cur, company_name):
    company_id = random.randint(1, 100)
    print(f"creating company with id: {company_id}...")

    try:
        cur.execute("insert into companies (id, name) values(?, ?);", (company_id, company_name))
    except sqlite3.IntegrityError:
        return None
    
    return company_id, company_name

And yes, again, before I get flamed for writing bad exception handling like this, I know, don't use a special return value to indicate an exception - that's what exceptions are for!

In [27]:
random.seed(0)
create_company(db_cur, "test")

creating company with id: 50...
creating company with id: 98...
creating company with id: 54...
creating company with id: 6...
creating company with id: 34...
creating company with id: 66...
creating company with id: 63...
creating company with id: 52...
creating company with id: 39...
creating company with id: 62...
creating company with id: 46...
creating company with id: 75...
creating company with id: 28...
creating company with id: 65...
creating company with id: 18...
creating company with id: 37...
creating company with id: 18...
creating company with id: 97...
creating company with id: 13...
creating company with id: 80...
creating company with id: 33...
creating company with id: 69...
creating company with id: 91...
creating company with id: 78...
creating company with id: 19...
creating company with id: 40...
creating company with id: 13...
creating company with id: 94...
creating company with id: 10...
creating company with id: 88...


(88, 'test')

As you can see, the function was retried (indefinitely, and with no delay) until it returned something other than `None`.

## Customizing Retry Conditions

Obviously everything I have shown you here can be applied to other situations, such as retrying API calls that fail due to rate limites, timeouts, etc.

I do not have access to a public API with a low rate limit to show you an example, but basically the idea is that your function making the API call will raise an HTTP exception. You can then specify the exact HTTP exception you want to perform retries with.

In general, rate limit HTTP exceptions are `429`, but various APIs may choose to report back a different error code, so you'll have to check the specific API you are working with.

The thing here, is that raising an http exception from your requests (using `raise_for_status`) will raise an HTTP exception for **any** HTTP error, not just a specific code, such as `429`.

To handle this you could either trap that exception and raise your own, and then retry based on that exception, or you could use Tenacity's setting to examine the exception message.

Let's see what an `HTTPError` in `requests` looks like.

In [29]:
import requests

try:
    result = requests.get("https://www.google.com/junk")
    result.raise_for_status()
except requests.HTTPError as ex:
    print(ex)
    print(ex.response.status_code)

404 Client Error: Not Found for url: https://www.google.com/junk
404


So, we basically want to do a retry based on the specific code found in the exception's `response.status_code` property.

This does not seem to be a pre-made option (to inspect the exception's properties to determine a retry), but it is easily achievable by creating a custom retry testing class for Tenacity, based on the `retry_base` abstract class. We just need to implement the `__call__` method that needs to return a boolean (`True` for retry, `False` otherwise.)

You can see the source code for `retry_base` [here](https://github.com/jd/tenacity/blob/main/tenacity/retry.py).

Let's see how to do that to handle a 429 specifically.

In [31]:
from tenacity.retry import retry_base

class retry_if_rate_limited(retry_base):
    def __call__(self, retry_state) -> bool:
        if retry_state.outcome.failed:
            ex = retry_state.outcome.exception()
            return isinstance(ex, requests.HTTPError) and ex.response.status_code == requests.codes.too_many_requests

For our function I'm going to simulate an HTTP 429 error:

In [32]:
@tenacity.retry(retry=retry_if_rate_limited(), stop=tenacity.stop_after_attempt(3))
def make_call():
    print("Trying API call...")
    ex = requests.models.Response()
    ex.status_code = requests.codes.too_many_requests
    raise requests.HTTPError(response=ex)

And let's try it out:

In [34]:
random.seed(0)

try:
    make_call()
except tenacity.RetryError as ex:
    print(ex)
    last_ex = ex.last_attempt.exception()
    print(last_ex.response.status_code)

Trying API call...
Trying API call...
Trying API call...
RetryError[<Future at 0x110dbd1d0 state=finished raised HTTPError>]
429


## Direct Retries without Decorators

There's plenty more you can do with the Tenacity library.

For example you don't need to wrap a function to retry code, you can do it right in the middle of your code using a context manager:

In [35]:
counter = 0

try:
    for attempt in tenacity.Retrying(
        retry=tenacity.retry_if_exception_type(ValueError), 
        stop=tenacity.stop_after_attempt(5)
    ):
        with attempt:
            counter += 1
            print(f"Attempt #{counter}")
            if counter < 3:
                raise ValueError()
            print("Success!")
except RetryError as ex:
    print(ex.last_attempt.exception())

Attempt #1
Attempt #2
Attempt #3
Success!


In [36]:
counter = 0

try:
    for attempt in tenacity.Retrying(
        retry=tenacity.retry_if_exception_type(ValueError), 
        stop=tenacity.stop_after_attempt(5)
    ):
        with attempt:
            counter += 1
            print(f"Attempt #{counter}")
            if counter < 3:
                raise ValueError()
            raise TypeError()
except Exception as ex:
    print(type(ex))

Attempt #1
Attempt #2
Attempt #3
<class 'TypeError'>


## Conclusion

Tenacity also allows you to hook up loggers for both before and after retry, as well as more options for retry conditions, delays (including jitter), and more.

In conclusion, this library is **extremely** handy, very **easy** to use, and offers a lot of **functionality**.

Happy Pythoning!